In [3]:
import numpy as np
import regions
import warnings
import glob
from astropy.io import fits
from astropy import stats
import pylab as pl
from astropy import units as u
from astropy import log
from astropy.coordinates import SkyCoord
from grid_strategy import strategies
from astropy.table import Table
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import simple_norm
import itertools
import dust_extinction
from astroquery.svo_fps import SvoFps
from astroquery.vizier import Vizier
from dust_extinction.averages import RRP89_MWGC, CT06_MWGC, F11_MWGC
from dust_extinction.parameter_averages import CCM89
import matplotlib as mpl

from matplotlib.path import Path
import matplotlib.patches as patches

#from filtering import get_fwhm

In [4]:
ls -l /orange/adamginsburg/jwst/cloudc/catalogs/*_merged_*.{fits,ecsv}

-rw-rw-r--+ 1 savannahgramze adamginsburg 1367195056 Jan 24 12:47 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_nsky0_merged_photometry_tables_merged.ecsv
-rw-rw-r--+ 1 savannahgramze adamginsburg  808182720 Jan 24 12:47 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_nsky0_merged_photometry_tables_merged.fits
-rw-rw-r--+ 1 savannahgramze adamginsburg 1412367372 Jan 24 13:12 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_nsky1_merged_photometry_tables_merged.ecsv
-rw-rw-r--+ 1 savannahgramze adamginsburg  830128320 Jan 24 13:12 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_nsky1_merged_photometry_tables_merged.fits
-rw-rw-r--+ 1 savannahgramze adamginsburg 1167488034 Jan 24 11:33 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_unweighted_merged_photometry_tables_merged.ecsv
-rw-rw-r--+ 1 savannahgramze adamginsburg  653892480 Jan 24 11:34 /orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_unweighted_merged_photometry_tables_merged.fits


In [5]:
merge_cat = Table.read('/orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_nsky1_merged_photometry_tables_merged.ecsv')

In [4]:
merge_cat

skycoord_ref,skycoord_ref_filtername,sep_f410m,id_f410m,skycoord_f410m,x_f410m,y_f410m,flux_f410m,passno_f410m,dx_f410m,dy_f410m,dflux_f410m,qf_f410m,rchi2_f410m,fracflux_f410m,fluxlbs_f410m,dfluxlbs_f410m,fwhm_f410m,spread_model_f410m,dspread_model_f410m,fluxiso_f410m,xiso_f410m,yiso_f410m,flags_f410m,sky_f410m,mask_f410m,flux_jy_f410m,eflux_jy_f410m,mag_ab_f410m,emag_ab_f410m,near_saturated_f410m_f410m,replaced_saturated_f410m,sep_f212n,id_f212n,skycoord_f212n,x_f212n,y_f212n,flux_f212n,passno_f212n,dx_f212n,dy_f212n,dflux_f212n,qf_f212n,rchi2_f212n,fracflux_f212n,fluxlbs_f212n,dfluxlbs_f212n,fwhm_f212n,spread_model_f212n,dspread_model_f212n,fluxiso_f212n,xiso_f212n,yiso_f212n,flags_f212n,sky_f212n,mask_f212n,flux_jy_f212n,eflux_jy_f212n,mag_ab_f212n,emag_ab_f212n,near_saturated_f212n_f212n,replaced_saturated_f212n,sep_f466n,id_f466n,skycoord_f466n,x_f466n,y_f466n,flux_f466n,passno_f466n,dx_f466n,dy_f466n,dflux_f466n,qf_f466n,rchi2_f466n,fracflux_f466n,fluxlbs_f466n,dfluxlbs_f466n,fwhm_f466n,spread_model_f466n,dspread_model_f466n,fluxiso_f466n,xiso_f466n,yiso_f466n,flags_f466n,sky_f466n,mask_f466n,flux_jy_f466n,eflux_jy_f466n,mag_ab_f466n,emag_ab_f466n,near_saturated_f466n_f466n,replaced_saturated_f466n,sep_f405n,id_f405n,skycoord_f405n,x_f405n,y_f405n,flux_f405n,passno_f405n,dx_f405n,dy_f405n,dflux_f405n,qf_f405n,rchi2_f405n,fracflux_f405n,fluxlbs_f405n,dfluxlbs_f405n,fwhm_f405n,spread_model_f405n,dspread_model_f405n,fluxiso_f405n,xiso_f405n,yiso_f405n,flags_f405n,sky_f405n,mask_f405n,flux_jy_f405n,eflux_jy_f405n,mag_ab_f405n,emag_ab_f405n,near_saturated_f405n_f405n,replaced_saturated_f405n,sep_f187n,id_f187n,skycoord_f187n,x_f187n,y_f187n,flux_f187n,passno_f187n,dx_f187n,dy_f187n,dflux_f187n,qf_f187n,rchi2_f187n,fracflux_f187n,fluxlbs_f187n,dfluxlbs_f187n,fwhm_f187n,spread_model_f187n,dspread_model_f187n,fluxiso_f187n,xiso_f187n,yiso_f187n,flags_f187n,sky_f187n,mask_f187n,flux_jy_f187n,eflux_jy_f187n,mag_ab_f187n,emag_ab_f187n,near_saturated_f187n_f187n,replaced_saturated_f187n,sep_f182m,id_f182m,skycoord_f182m,x_f182m,y_f182m,flux_f182m,passno_f182m,dx_f182m,dy_f182m,dflux_f182m,qf_f182m,rchi2_f182m,fracflux_f182m,fluxlbs_f182m,dfluxlbs_f182m,fwhm_f182m,spread_model_f182m,dspread_model_f182m,fluxiso_f182m,xiso_f182m,yiso_f182m,flags_f182m,sky_f182m,mask_f182m,flux_jy_f182m,eflux_jy_f182m,mag_ab_f182m,emag_ab_f182m,near_saturated_f182m_f182m,replaced_saturated_f182m,flux_jy_410m405,mag_ab_410m405,flux_jy_405m410,mag_ab_405m410,flux_jy_182m187,mag_ab_182m187,flux_jy_187m182,mag_ab_187m182
"deg,deg",,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,deg,,"deg,deg",,,,,,,,,,,,,,,,,,,,,,Jy,Jy,,,,,Jy,Jy,Jy,Jy,Jy,Jy,Jy,Jy
SkyCoord,str5,float64,int64,SkyCoord,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int32,float32,bool,float32,float32,float32,float32,bool,bool,float64,int64,SkyCoord,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int32,float32,bool,float32,float32,float32,float32,bool,bool,float64,int64,SkyCoord,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int32,float32,bool,float32,float32,float32,float32,bool,bool,float64,int64,SkyCoord,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int32,float32,bool,float32,float32,float32,float32,bool,bool,float64,int64,SkyCoord,float64,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,int32,float32,bool,float32,float32,float32,float32,bool,boo

In [6]:
merge_cat.columns

<TableColumns names=('skycoord_ref','skycoord_ref_filtername','sep_f410m','id_f410m','skycoord_f410m','x_f410m','y_f410m','flux_f410m','passno_f410m','dx_f410m','dy_f410m','dflux_f410m','qf_f410m','rchi2_f410m','fracflux_f410m','fluxlbs_f410m','dfluxlbs_f410m','fwhm_f410m','spread_model_f410m','dspread_model_f410m','fluxiso_f410m','xiso_f410m','yiso_f410m','flags_f410m','sky_f410m','mask_f410m','flux_jy_f410m','eflux_jy_f410m','mag_ab_f410m','emag_ab_f410m','near_saturated_f410m_f410m','replaced_saturated_f410m','sep_f212n','id_f212n','skycoord_f212n','x_f212n','y_f212n','flux_f212n','passno_f212n','dx_f212n','dy_f212n','dflux_f212n','qf_f212n','rchi2_f212n','fracflux_f212n','fluxlbs_f212n','dfluxlbs_f212n','fwhm_f212n','spread_model_f212n','dspread_model_f212n','fluxiso_f212n','xiso_f212n','yiso_f212n','flags_f212n','sky_f212n','mask_f212n','flux_jy_f212n','eflux_jy_f212n','mag_ab_f212n','emag_ab_f212n','near_saturated_f212n_f212n','replaced_saturated_f212n','sep_f466n','id_f466n','sk